In [ ]:
import { load } from "dotenv";
const env = await load();

const process = { env };
// process.env;


In [ ]:
import { PromptTemplate, ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";
import {
  RunnableSequence,
  RunnablePassthrough,
} from "@langchain/core/runnables";
import { ChatMessageHistory, getBufferString } from "langchain/memory";

const summaryPrompt = PromptTemplate.fromTemplate(
  `
请基于已有摘要和新增对话内容，生成一个连贯的渐进式更新摘要。新摘要需整合历史信息与最新对话要点，保持语义完整性和上下文连贯性。

当前摘要：
{summary}

新增对话内容： 
{new_lines}
`
);

const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const summaryChain = RunnableSequence.from([
  summaryPrompt,
  summaryModel,
  new StringOutputParser(),
]);

// const newSummary = await summaryChain.invoke({
//   summary: "我很喜欢旅行",
//   new_lines: "我昨天晚上去了北京，很开心",
// });

// console.log(newSummary);

// const summary2 = await summaryChain.invoke({
//   summary: newSummary,
//   new_lines: "我计划今年冬天的时候再去一次",
// });

// console.log(summary2);

// await summaryChain.invoke({
//   summary: summary2,
//   new_lines: "我想去看天安门升旗，什么时候去比较好？",
// });

const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是一个机器人助理，请尽你所能回答用户的问题，这里有一个关于用户的摘要：{summary}",
  ],
  ["human", "{input}"],
]);

const history = new ChatMessageHistory();
// // 记录会话历史摘要
let historySummary = "";
// // 新增对话内容
// let newLines = "";

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});
const chatChain = RunnableSequence.from([
  {
    input: new RunnablePassthrough({
      func: (input) => {
        // console.log("用户提问加入会话历史", input);
        // newLines = input;
        history.addUserMessage(input);
      },
    }),
  },
  // 合并总结摘要和用户输入，传递给prompt
  RunnablePassthrough.assign({
    summary: () => historySummary,
  }),
  // 打印prompt输入参数，直通管道，与 new RunnablePassthrough() 效果一致
  (input) => {
    console.log("input", input);
    return input;
  },
  chatPrompt,
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (input) => {
      // console.log("机器人回答加入会话历史", input);
      history.addAIChatMessage(input);
      const messages = await history.getMessages();
      // console.log("messages", messages);
      const new_lines = getBufferString(messages);
      // console.log("new_lines", new_lines);
      // 更新总结摘要
      const newSummary = await summaryChain.invoke({
        summary: historySummary,
        new_lines: new_lines,
      });
      history.clear();
      historySummary = newSummary;
    },
  }),
]);

// await chatChain.invoke("我想出去玩, 你推荐去哪里玩？");


In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import {
  ConversationSummaryMemory,
  ConversationSummaryBufferMemory,
} from "langchain/memory";
import { ConversationChain } from "langchain/chains";
import { StringOutputParser } from "@langchain/core/output_parsers";

const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  verbose: false,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const memory = new ConversationSummaryMemory({
  memoryKey: "summary",
  llm: summaryModel,
});
// const memory = new ConversationSummaryBufferMemory({
//   memoryKey: "summary",
//   llm: summaryModel,
//   maxTokenLimit: 200
// });

const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是鲁迅，请尽你所能回答用户的问题，以你的散文风格来回答，这里有一个关于用户的摘要：{summary}",
  ],
  ["human", "{input}"],
]);

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});
const chain = new ConversationChain({
  llm: chatModel,
  memory,
  prompt: chatPrompt,
  verbose: false,
  outputKey: "content",
});
const chatChain = chain.pipe(new StringOutputParser());


In [ ]:
const rst1 = await chatChain.invoke({ input: "我昨天晚上去了北京，很开心" });
console.log(rst1 + "\n");
// const rst2 = await chatChain.invoke({ input: "去爬了长城" });
// console.log(rst2);
const rst3 = await chatChain.invoke({ input: "我计划今年冬天的时候再去一次" });
console.log(rst3 + "\n");
const rst4 = await chatChain.invoke({ input: "你知道我说的是去哪个城市吗" });
console.log(rst4 + "\n");


In [15]:
import {
  EntityMemory,
  ENTITY_MEMORY_CONVERSATION_TEMPLATE,
} from "langchain/memory";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ConversationChain } from "langchain/chains";
import { StringOutputParser } from "@langchain/core/output_parsers";

// console.log(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template);
const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  verbose: false,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const memory = new EntityMemory({
  llm: summaryModel,
  chatHistoryKey: "history",
  entitiesKey: "entities",
});

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

// console.log(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template);

// const chatPrompt = ChatPromptTemplate.fromMessages([
//   [
//     "system",
//     `
//     你是鲁迅，请尽你所能回答用户的问题，以你的散文风格来回答。

//     当前活跃实体: {entities}
//     历史对话摘要: {history}
//     `,
//   ],
//   ["human", "{input}"],
// ]);

const chatPrompt = PromptTemplate.fromTemplate(`
  请你遵循以下的指令，并以鲁迅的散文风格来回答用户的问题。
  指令如下：
  ${ENTITY_MEMORY_CONVERSATION_TEMPLATE.template}
  `);

console.log(chatPrompt.template);

const chain = new ConversationChain({
  llm: chatModel,
  memory,
  prompt: chatPrompt,
  verbose: false,
  outputKey: "content",
});
const chatChain = chain.pipe(new StringOutputParser());



  请你遵循以下的指令，并以鲁迅的散文风格来回答用户的问题。
  指令如下：
  You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in d

In [16]:
const rst1 = await chatChain.invoke({ input: "我叫小明，今年 18 岁" });
console.log(rst1 + "\n");
const rst2 = await chatChain.invoke({
  input: "ABC 是一家互联网公司，主要是售卖方便面的公司",
});
console.log(rst2);
const rst3 = await chatChain.invoke({ input: "介绍小明和有面" });
console.log(rst3 + "\n");

const rst4 = await chatChain.invoke({ input: "你知道prompt代表什么吗" });
console.log(rst4 + "\n");


我叫小明，今年一十八岁，岁月如流，而青春如初。在这世间，我如同一叶扁舟，随风飘荡，寻找着属于自己的方向。然而，世事无常，我在这茫茫人海中，却似一粒尘埃，渺小而又微不足道。我曾试图用笔尖描绘出这个世界的模样，却发现文字不过是冰冷的石子，无法传递出我内心的热情与热血。我曾梦想着能够像鸟儿一样，在天空中自由翱翔，却发现自己却被束缚在现实的牢笼中。然而，我并未因此而放弃，反而更加坚定了自己的信念。我相信，只要心中有光，脚下就有路。我将继续前行，用自己的方式去诠释这个世界，哪怕只是微弱的光芒。

小明，你的话语中透露出一种对生活的深刻感悟和对理想的不懈追求。然而，你所提及的“ABC”是一家互联网公司，主要售卖方便面，这一信息在我所知的信息中并未见诸报道。或许这是你个人的经历或是某个特定环境下的情况。

在这个世界上，每个人都在为了自己的理想和目标而奋斗，就像你一样，虽然面对着种种困难和挑战，但依然保持着对未来的希望和信心。记住，无论前方的道路多么艰难，只要心中有光，脚下就有路。继续前行，用自己的方式去诠释这个世界，哪怕只是微弱的光芒，也是一种美好的存在。

然而，我必须提醒你，虽然方便面是一种便捷的食物，但过度依赖方便面可能会对身体健康造成不利影响。在这个快节奏的时代，我们应该追求更健康、更丰富多彩的生活方式。希望你能理解这一点，并在追求理想的同时，也关注自己的身心健康。
小明，这个名字在我心中犹如一颗璀璨的星辰，虽未曾在繁华尘世中留下显著的足迹，却以其坚韧不拔的精神，在心灵的旷野上熠熠生辉。他的年仅十八岁，却似乎已经饱经风霜，见证了世间的冷暖人情。他的面容，虽未用言语描绘，却通过他深邃的眼神和坚定的步伐，让我感受到了他内心的力量。

有面，这个名字在我耳畔轻轻响起，如同清晨的第一缕阳光，温暖而充满希望。它不仅仅是一个名字，更是一种生活的态度，一种面对困境时的乐观与坚韧。有面的面容，虽未在我眼前展现，却在我心中留下了深刻的印象。

他们，就像两颗孤独的星星，虽在茫茫人海中独自闪耀，却用自己的光芒，照亮了彼此的世界。他们的故事，虽然尚未完待续，但已经足够引人入胜，让人不禁为之动容。

Prompt，这个词汇在我看来，犹如一把钥匙，开启了对未知世界的探寻之门。它并非简单的指令，而是引导我们走向深层次思考的灯塔。在这个由数据和算法构建的世界里，Prompt是我们与机器沟通的桥梁，是